In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils
import os
import pandas as pd
from pandas import DataFrame, Series, Timestamp, Index
from tqdm import tqdm

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
from numpy import load
import plotly.graph_objects as go
from plotly.graph_objects import Figure

IMPORT FUNCTIONS TO TRANSFORM DF AND CALCULATE MAPE

In [4]:
from inference import mape
from inference import utils as infutils
from script_download_data import preprocess_sorgenia

In [5]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [6]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [7]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

Selecting GPU ID=0


In [8]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_final_cop.csv'

In [9]:
config = ExperimentConfig('sorgenia_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [ ]:
# extract data into DataFrame
# raw_data: DataFrame = pd.read_csv(file_path)
raw_data: DataFrame = preprocess_sorgenia('copernicus', config, True)

In [ ]:
raw_data.head()

# START WITH MODEL1

Wind Forecasts as known input

In [ ]:
formatter: SorgeniaFormatter = config.make_data_formatter()

In [ ]:
data_csv_path: str = config.data_csv_path

# SPLIT DATA

In [ ]:
train, valid, test = formatter.split_data(raw_data)

In [ ]:
# Sets up default params
fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [ ]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

In [ ]:
predictions_path: str = r'C:\Users\Lorenzo\savedmodels_sorgenia_wind_known'

In [ ]:
def compute_predictions(test: DataFrame, opt_manager: HyperparamOptManager, formatter: SorgeniaFormatter, tf_config: ConfigProto):
    print("*** Running tests ***")
    tf1.reset_default_graph()
    with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
        tf1.keras.backend.set_session(sess)
        params: Dict = opt_manager.get_next_parameters()
        params['exp_name'] = 'sorgenia_wind'
        params['data_folder'] = os.path.abspath(os.path.join(data_csv_path, os.pardir))
        model = TemporalFusionTransformer(params, use_cudnn=False)
        params.pop('exp_name', None)
        params.pop('data_folder', None)
        # load model
        model.load(opt_manager.hyperparam_folder, use_keras_loadings=True)

    #     print("Computing best validation loss")
    #     val_loss: Series = model.evaluate(valid)

        print("Computing test loss")
        output_map: Dict = model.predict(test, return_targets=True)
        print(f"Output map returned a dict with keys {output_map.get('p50').shape}")
        targets: DataFrame = formatter.format_predictions(output_map["targets"])
        p50_forecast: DataFrame = formatter.format_predictions(output_map["p50"])
        p90_forecast: DataFrame = formatter.format_predictions(output_map["p90"])

        # save all
        print("saving predictions and targets")
        targets.to_csv(os.path.join(opt_manager.hyperparam_folder, "targets.csv"), index=False)
        p50_forecast.to_csv(os.path.join(opt_manager.hyperparam_folder, "p50.csv"), index=False)
        p90_forecast.to_csv(os.path.join(opt_manager.hyperparam_folder, "p90.csv"), index=False)

        def extract_numerical_data(data: DataFrame) -> DataFrame:
            """Strips out forecast time and identifier columns."""
            return data[[
                col for col in data.columns
                if col not in {"forecast_time", "identifier"}
            ]]

        p50_loss = utils.numpy_normalised_quantile_loss(
                extract_numerical_data(targets), extract_numerical_data(p50_forecast),
                0.5)
        p90_loss = utils.numpy_normalised_quantile_loss(
            extract_numerical_data(targets), extract_numerical_data(p90_forecast),
            0.9)

        tf1.keras.backend.set_session(default_keras_session)

    print()
    print("Normalised Quantile Loss for Test Data: P50={}, P90={}".format(
        p50_loss.mean(), p90_loss.mean()))

In [ ]:
compute_predictions(test, opt_manager, formatter, tf_config)

# OPEN PREDICTIONS AND TARGETS FILES

In [ ]:
from inference import evaluate

In [ ]:
df_mape = evaluate.evaluate(opt_manager.hyperparam_folder, 'copernicus_forecasts_model', 700)

# PLOT MAPE BOXPLOT

In [ ]:
fig: Figure = evaluate.boxplotter(df_mape)
fig.show()